In [4]:
import numpy as np

In [326]:
class Matrix:
    def __init__(self, data, requires_grad = True):
        self.data = data
        self.requires_grad = requires_grad
        self.dtype = self.data.dtype
        self.shape = self.data.shape
        if requires_grad:
            self.grad = np.zeros_like(self.data, dtype=np.float32)
        else:    
            self.grad = None
        if self.requires_grad:
            # TODO
            pass

    def __str__(self):
        result = 'matrix(' + str(self.data)
        return result.replace('\n', '\n' + ' ' * len('matrix(')) + (f', requires_grad={self.requires_grad}') + ')'

    def __repr__(self):
        return self.__str__()
    
    def set_dtype(self, dtype):
        self.data = np.matrix(self.data, dtype=dtype)
        return self

    def float(self):
        return self.set_dtype(np.float32)
    
    def sum(self):
        # self._accumulate_add()
        return Matrix(self.data.sum(), requires_grad=True)
    
    def _zero_grad(self):
        self.grad = np.zeros_like(self.data, dtype=self.dtype)
    
    def _accumulate_add(self):
        self.grad += 1
    
    def _accumulate_mul(self, other):
        for i in range(self.shape[0]):
            self.grad[:, i] += other.data[i, :].sum()
        
    def _accumulate_rmul(self, other):
        for i in range(self.shape[0]):
            self.grad[i, :] += other.data[:, i].sum()
    
    def __add__(self, other):
        assert isinstance(other, Matrix)
        
        if self.data.shape != other.data.shape:
            pass # TODO
        
        self._accumulate_add()
        other._accumulate_add()
        return Matrix(self.data + other.data, requires_grad=self.requires_grad)
    
    def __mul__(self, other):
        assert isinstance(other, Matrix)
        
        self._accumulate_mul(other)
        other._accumulate_rmul(self)
        return Matrix(np.matmul(self.data, other.data), requires_grad=self.requires_grad)

    def __rmul__(self, other):
        # Probably not necessary
        assert isinstance(other, Matrix)
        self._accumulutate_rmul(other)
        return Matrix(np.matmul(other.data, self.data), requires_grad=self.requires_grad)

# Example

In [341]:
# m = np.matrix([1,2,3,4,5,6,7,8,9]).reshape(3,3)
h = 1
m = np.matrix([1, 2+h,
               3, 4]).reshape(2,2)
n = np.matrix([1, 2,
               3, 4]).reshape(2,2)
m = Matrix(m).float()
n = Matrix(n)
z = m * n
loss = z.sum()
loss

matrix(61.0, requires_grad=True)

In [335]:
m.grad

matrix([[3., 7.],
        [3., 7.]], dtype=float32)

In [256]:
m = np.matrix([1,2])
n = np.matrix([1,2,3,4]).reshape(2,2)
a = Matrix(m)
b = Matrix(n)
a * b

matrix([[ 7 10]], requires_grad=True)

In [257]:
a.grad

matrix([[3., 0.]], dtype=float32)